### Porque utilizar um modelo de regressão linear?

Modelos de regressão linear são ótimos para esse problema por ele ter uma natureza linear e de prever variáveis contínuas, por exemplo: pode ter uma proporcionalidade entre cargo e função em relação ao salário 

### Dataprep

#### Ler o arquivo fonte

In [31]:
import pandas as pd

## ler o csv bruto
df = pd.read_csv('./salarios_recife_outubro_2013.csv', sep=',', header=0)

df

,ORGAO,NOME,CATEGORIA,CARGO,FUNCAO,R$SALARIO
0,SECRETARIA DE EDUCACAO,9/16/13,ESTATUTARIO,TECNICO EDUCACIONAL,NaN,R$1902.24
1,SECRETARIA DE EDUCACAO,GOV. PERNAMBUCO **** CONSISTHR **...,ESTATUTARIO,ASSIST ADMINIST EDUCACIONAL,NaN,R$2667.97
2,SECRETARIA DE EDUCACAO,ARNEY SPINELI GUIMARAES,ESTATUTARIO,TECNICO EDUCACIONAL,NaN,R$2302.24
3,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZACAO,ARNEY SPINELI GUIMARAES,CTD - CONTRATO TEMPO DETERMINADO,PSICOLOGO - CTD,NaN,R$1770
4,SECRETARIA DE EDUCACAO,ITA DE CASSIA FISCHER DEL PINO,ESTATUTARIO,PROFESSOR,NaN,R$2325.26
...,...,...,...,...,...,...
133907,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,SIMONE VALERIA DE GUSMAO SANTOS,CTD - CONTRATO TEMPO DETERMINADO,ASSISTENTE ADMINISTRATIVO CTD,NaN,R$1100
133908,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,THAIS SOARES FONSECA,COMISSIONADO,ASSESS SUP AO SEC-ASS NAT TEC,ASSESS SUP AO SEC-ASS NAT TEC,R$4429.61
133909,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,VIVIANE DA SILVA LUCENA PATRIOTA,CTD - CONTRATO TEMPO DETERMINADO,TECNICO NA AREA SOCIAL - CTD,NaN,R$2200
133910,ITERPE - INSTITUTO DE TERRAS E REFORMA AGRARIA...,WAGNER LEITE DE OLIVEIRA,CTD - CONTRATO TEMPO DETERMINADO,TECNICO AGRICOLA - CTD,NaN,R$1635


#### Lidar com nulos

In [32]:
df.apply(lambda x : sum(x.isnull()), axis=0)

ORGAO            0
NOME             0
CATEGORIA        0
CARGO            0
FUNCAO       91556
R$SALARIO        0
dtype: int64

#### Arrumando o valor do salário

In [33]:
df['R$SALARIO'] = df['R$SALARIO'].apply(lambda x: float(x.split('R$')[1]))

df['R$SALARIO']

0         1902.24
1         2667.97
2         2302.24
3         1770.00
4         2325.26
           ...   
133907    1100.00
133908    4429.61
133909    2200.00
133910    1635.00
133911    4590.00
Name: R$SALARIO, Length: 133912, dtype: float64

#### Transformar categorias em números

In [34]:
from sklearn.preprocessing import LabelEncoder

df = df.drop(axis=1, columns=['NOME']) # o nome é desnecessário para prever o valor de salário

le = LabelEncoder()

for i in df.columns:
    df[i] = le.fit_transform(df[i])

df

,ORGAO,CATEGORIA,CARGO,FUNCAO,R$SALARIO
0,48,4,811,764,3446
1,48,4,188,764,6180
2,48,4,811,764,4954
3,59,3,666,764,2941
4,48,4,640,764,5040
...,...,...,...,...,...
133907,26,3,215,764,801
133908,26,1,162,76,11128
133909,26,3,840,764,4566
133910,26,3,785,764,2396


### Treinamento do modelo

#### Separando os dados

In [35]:
df_vars_indep = df.drop('R$SALARIO', axis=1)

df_alvo = df[['R$SALARIO']]

from sklearn.model_selection import train_test_split

vars_train, vars_test, res_train, res_test = train_test_split(df_vars_indep, df_alvo, test_size=0.3, random_state=777)

#### Treinando o modelo

In [36]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(vars_train, res_train)

res_prev = model.predict(vars_train) # previsões geradas

res_prev

array([[7566.2273344 ],
       [5082.02161825],
       [5478.28465802],
       ...,
       [7560.31602367],
       [7560.31602367],
       [5572.84718512]])

#### Medindo performance do modelo

##### Usando dados de treino

In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

predict_train = model.predict(vars_train)

print(mean_absolute_error(res_train, predict_train)) # medir a média de erro de variável contínua 
print(mean_squared_error(res_train, predict_train)) # grau de intensidade do erro
print(mean_absolute_error(res_train, predict_train) / res_train.mean()) # percent de diferença da média do real com o da previsão

3676.4130303394086
20390448.902625635
R$SALARIO    0.641385
dtype: float64


O modelo errou apenas 36% dos casos, errando em média cerca de R$3686,03 o valor de salário e podendo errar de R$4515,57 para cima ou para baixo. A performance não é satisfatória por esperar no mínimo 70%

##### Usando dados de teste

In [38]:
predict_test = model.predict(vars_test)

print(mean_absolute_error(res_test, predict_test))
print(mean_squared_error(res_test, predict_test))
print(mean_absolute_error(res_test, predict_test) / res_test.mean())

3672.1487265491805
20321044.83909753
R$SALARIO    0.641214
dtype: float64


Os resultado das previsões em comparação com os dados de teste demonstram que não há um overfitting ou underfitting dos dados

### Conclusão

O modelo possuí uma precisão abaixo do esperado (65%), para melhorar sua performance seria necessário mais dados, sejam por mais colunas para aumentar a quantidade de variáveis independentes ou mais linhas para aumentar os possíveis valores para a variável dependente